<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Modelo de lenguaje con tokenización por palabras

### Consigna
- Seleccionar un corpus de texto sobre el cual entrenar el modelo de lenguaje.
- Realizar el pre-procesamiento adecuado para tokenizar el corpus, estructurar el dataset y separar entre datos de entrenamiento y validación.
- Proponer arquitecturas de redes neuronales basadas en unidades recurrentes para implementar un modelo de lenguaje.
- Con el o los modelos que consideren adecuados, generar nuevas secuencias a partir de secuencias de contexto con las estrategias de greedy search y beam search determístico y estocástico. En este último caso observar el efecto de la temperatura en la generación de secuencias.


### Sugerencias
- Durante el entrenamiento, guiarse por el descenso de la perplejidad en los datos de validación para finalizar el entrenamiento. Para ello se provee un callback.
- Explorar utilizar SimpleRNN (celda de Elman), LSTM y GRU.
- rmsprop es el optimizador recomendado para la buena convergencia. No obstante se pueden explorar otros.


In [3]:
import random
import io
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy

### Datos
Utilizaremos como dataset el libro de Viaje al Centro de la Tierra de Julio Verne.

In [4]:
# Descargar la carpeta de dataset
import os
import platform

directory = './datasets/text_corpus'

if os.access(directory, os.F_OK) is False:
    os.makedirs(directory)
if os.access('desafio_2.zip', os.F_OK) is False:
    if platform.system() == 'Windows':
        !curl https://raw.githubusercontent.com/simon-rodriguez/CEIA_procesamiento_lenguaje_natural/main/datasets/text_corpus/desafio_2.zip -O {directory}/desafio_2.zip

    else:
        !wget -O {directory}/desafio_2.zip https://github.com/simon-rodriguez/CEIA_procesamiento_lenguaje_natural/raw/main/datasets/text_corpus/desafio_2.zip

    !unzip -q {directory}/desafio_2.zip -d {directory}

else:
    print("El dataset ya se encuentra descargado")

--2024-10-03 14:28:37--  https://github.com/simon-rodriguez/CEIA_procesamiento_lenguaje_natural/raw/main/datasets/text_corpus/desafio_2.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/simon-rodriguez/CEIA_procesamiento_lenguaje_natural/main/datasets/text_corpus/desafio_2.zip [following]
--2024-10-03 14:28:38--  https://raw.githubusercontent.com/simon-rodriguez/CEIA_procesamiento_lenguaje_natural/main/datasets/text_corpus/desafio_2.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 981510 (959K) [application/zip]
Saving to: ‘./datasets/text_corpus/desafio_2.zip’

./datasets/text_cor 100%[===

In [5]:
# Posibles libros
os.listdir("./datasets/text_corpus")

['the_adventures_of_sherlock_holmes.txt',
 'viaje_al_centro_de_la_tierra_sentences_fix.txt',
 'viaje_al_centro_de_la_tierra.txt',
 'viaje_al_centro_de_la_tierra_sentences.txt',
 'the_adventures_of_sherlock_holmes_sentences.txt',
 'desafio_2.zip',
 'el_arte_de_la_guerra.txt']

In [6]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
# El libro está preprocesado para que cada línea sea una oración del libro (determinada por el punto al final de la oracion)
df_viaje = pd.read_csv('./datasets/text_corpus/viaje_al_centro_de_la_tierra_sentences_fix.txt', sep='/n', header=None)
df_viaje.head()

<ipython-input-6-f54effbf10b4>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_viaje = pd.read_csv('./datasets/text_corpus/viaje_al_centro_de_la_tierra_sentences_fix.txt', sep='/n', header=None)


,0
0,Viaje al Centro de la Tierra
1,Julio Verne
2,Capítulo 1
3,"El domingo 24 de mayo de 1863, mi tío, el prof..."
4,"Marta, su excelente criada, se preocupó sobrem..."


Consideraremos que cada secuencia para este dataset es una oración, delimitada por un punto final.

Una de las primeras decisiones que hay que tomar es el tamaño de contexto de tokens máximo que puede consumir el modelo. Este podría ser un hiperparámetro del problema.

Para elegir el tamaño de contexto máximo para este problema se puede explorar el dataset, para ello:
- se consideran las palabras como términos.
- se segmentará el texto de todos las oraciones del dataset y ses explorará la cantidad de términos presentes.



In [7]:
print("Cantidad de documentos:", df_viaje.shape[0])

Cantidad de documentos: 4061


### Elegir el tamaño del contexto

In [8]:
# Usaremos las utilidades de procesamiento de textos y secuencias de Keras
from tensorflow.keras.preprocessing.text import Tokenizer # equivalente a ltokenizer de nltk
from tensorflow.keras.preprocessing.text import text_to_word_sequence # equivalente a word_tokenize de nltk
from tensorflow.keras.utils import pad_sequences # se utilizará para padding


In [9]:
# cada oracion la guardamos en una lista
text = list(df_viaje.loc[:,0])
text

['Viaje al Centro de la Tierra',
 'Julio Verne',
 'Capítulo 1',
 'El domingo 24 de mayo de 1863, mi tío, el profesor Lidenbrock, entró rápidamente a su hogar, situado en el número 19 de la König‑Strasse, una de las calles más tradicionales del barrio antiguo de Hamburgo.',
 'Marta, su excelente criada, se preocupó sobremanera, creyendo que se había retrasado, pues apenas empezaba a cocinar la comida en el hornillo.',
 '“Bueno” — pensé para mí— , “si mi tío viene con hambre, se va a armar la de San Quintín; porque no conozco a otro hombre de menos paciencia”.',
 '— ¡Tan temprano y ya está aquí el señor Lidenbrock! — exclamó la pobre Marta, con arrebol, entreabriendo la puerta del comedor.',
 '— Sí, Marta; pero tú no tienes la culpa de que la comida no esté lista todavía, porque es temprano, aún no son las dos.',
 'Acaba de dar la media hora en San Miguel.',
 '— ¿Y por qué ha venido tan pronto el señor Lidenbrock?',
 '— Él lo explicará, seguramente.',
 '— ¡Ahí viene! Yo me escapo.',
 'Se

In [10]:
# segmentamos el texto con la utilidad de Keras
segmented_sentences = [text_to_word_sequence(sentence) for sentence in text]

In [11]:
text[0]

'Viaje al Centro de la Tierra'

In [12]:
segmented_sentences[0]

['viaje', 'al', 'centro', 'de', 'la', 'tierra']

In [13]:
# calculamos la longitud de cada secuencia
length_sentences = [len(sentence) for sentence in segmented_sentences]

(array([732., 976., 793., 583., 403., 254., 139.,  73.,  38.,  35.,  14.,
          7.,   4.,   2.,   3.,   1.,   2.,   1.,   0.,   1.]),
 array([  0. ,   6.2,  12.4,  18.6,  24.8,  31. ,  37.2,  43.4,  49.6,
         55.8,  62. ,  68.2,  74.4,  80.6,  86.8,  93. ,  99.2, 105.4,
        111.6, 117.8, 124. ]),
 <BarContainer object of 20 artists>)

In [14]:
# podemos ver su distribución (x=cantidad de segmentos; y= frecuencia)
plt.hist(length_sentences,bins=20)

In [15]:
# a partir de la distribución de longitudes de secuencias elegimos algún criterio
# para determinar el máximo tamaño de contexto. En este caso es un percentil, pero
# otros criterios también pueden ser válidos con la justificación adecuada.
# También puede ser una selección "a mano"

# el -1 es porque el último token será el target
max_context_size_per = int(np.percentile(length_sentences, 90)-1)

max_context_size_avg = int(np.ceil(np.mean(length_sentences))) # criterio de media
max_context_size_med = int(np.ceil(np.median(length_sentences))) # criterio de mediana


print(f'max_context_size_per: {max_context_size_per}')
print(f'max_context_size_avg: {max_context_size_avg}')
print(f'max_context_size_med: {max_context_size_med}')

max_context_size_per: 34
max_context_size_avg: 18
max_context_size_med: 15


In [16]:
max_context_size = max_context_size_per

print(f'max_context_size: {max_context_size}')

max_context_size: 34


###  Tokenizar

In [17]:
# instanciamos el tokenizador
tok = Tokenizer()

# El tokenizer "aprende" las palabras que se usaran
# Se construye (fit) una vez por proyecto, se aplica N veces (tal cual un encoder)
# El token 0 es reservado y no es asignado. Se utiliza para designar a palabras
# fuera del vocabulario aprendido
tok.fit_on_texts(segmented_sentences)

# Convertimos las palabras a números
# entran palabras -> salen números
tokenized_sentences = tok.texts_to_sequences(segmented_sentences)

In [18]:
tokenized_sentences[0]

[126, 21, 171, 1, 2, 62]

In [19]:
segmented_sentences[0]

['viaje', 'al', 'centro', 'de', 'la', 'tierra']

### Organizando y estructurando el dataset

En este punto tenemos en la variable `tokenized_sentences` los versos tokenizados. Vamos a quedarnos con un conjunto de validación que utilizaremos para medir la calidad de la generación de secuencias con la métrica de Perplejidad.

In [22]:
tokenized_sentences_train, tokenized_sentences_val, _, _ = train_test_split(tokenized_sentences, tokenized_sentences, test_size=0.2, random_state=42)

Vamos a splitear las oraciones que tienen tamaño mayor al contexto máximo, para generarnos más secuencias de entrenamiento. Este paso puede obviarse si el tamaño de contexto máximo es muy grande.

Por ejemplo, si tenemos el texto *La bella y graciosa moza marchóse a lavar la ropa* y nuestro contexto es de 5 palabras, tendremos:

- *La bella y graciosa moza*
- *bella y graciosa moza marchóse*
- *y graciosa moza marchóse a*
- *graciosa moza marchóse a lavar*
- *moza marchóse a lavar la*
- *marchóse a lavar la ropa*

In [23]:
tok_sent = []

for sent in tokenized_sentences_train:

  # si la secuencia tiene más términos que el tamaño de contexto máximo,
  # armo varias sub-secuencias de tamaño máximo
  if len(sent) > (max_context_size+1):
    extra = len(sent)-(max_context_size+1) + 1
    for i in range(extra):
      tok_sent.append(sent[i:i+max_context_size+1])
  else: # si la secuencia tiene menos términos el tamaño de contexto máximo, dejo la secuencia como está
    tok_sent.append(sent)


In [24]:
len(tok_sent)

6726

**Debido al tamaño del corpus, vamos a obviar esta parte (no hay suficiente memoria en el colab para procesarlo).**

Ahora aumentamos los datos aprovechando que de una secuencia grande se pueden generar varias más pequeñas:

- *La hermosa casa en el prado*
- *La hermosa*
- *La hermosa casa*
- *La hermosa casa en*
- *La hermosa casa en el*
- *La hermosa casa en el prado*

In [72]:
aumentar = False

In [73]:
if aumentar:
  print("aumentado")
  ##################################################
  # PARTE NO REALIZADA DEBIDO AL TAMAÑO DEL CORPUS #
  ##################################################
  tok_sent_augm = []

  for sent in tok_sent:

    # generamos todas las sub-secuencias
    subseq = [sent[:i+2] for i in range(len(sent)-1)]
    # en esta línea paddeamos al tamaño de contexto máximo
    tok_sent_augm.append(pad_sequences(subseq, maxlen=max_context_size+1, padding='pre'))

  # finalmente concatenamos todas las secuencias en un único array de numpy
  train_seqs = np.concatenate(tok_sent_augm, axis=0)
else:
  print("no aumentado")

  tok_sent_noaugm = []

  # en esta línea paddeamos al tamaño de contexto máximo
  tok_sent_noaugm.append(pad_sequences(tok_sent, maxlen=max_context_size+1, padding='pre'))

  # concatenamos todas las secuencias (no aumentadas) en un único array de numpy
  train_seqs = np.concatenate(tok_sent_noaugm, axis=0)




no aumentado


In [74]:
train_seqs.shape

(6726, 35)

In [75]:
# y de aquí sacamos las entradas y los targets que consumirá nuestro sistema en
# tiempo de entrenamiento
X = train_seqs[:,:-1]
y = train_seqs[:,1:]

Nótese que estamos estructurando el problema de aprendizaje como *many-to-many*:

Entrada: secuencia de tokens [$x_0$, $x_1$, ..., $x_N$]

Target: secuencia de tokens [$x_1$, $x_2$, ..., $x_{N+1}$]

De manera que la red tiene que aprender que su salida deben ser los tokens desplazados en una posición y un nuevo token predicho (el N+1).

La ventaja de estructurar el aprendizaje de esta manera es que para cada token de target se propaga una señal de gradiente por el grafo de cómputo recurrente, que es mejor que estructurar el problema como *many-to-one* en donde sólo una señal de gradiente se propaga.

In [76]:
y.shape

(6726, 34)

In [77]:
# Palabras del vocabulario
tok.index_word

{1: 'de',
 2: 'la',
 3: 'que',
 4: 'el',
 5: 'y',
 6: 'a',
 7: 'en',
 8: '—',
 9: 'no',
 10: 'un',
 11: 'las',
 12: 'se',
 13: 'los',
 14: 'del',
 15: 'por',
 16: 'mi',
 17: 'una',
 18: 'con',
 19: 'me',
 20: 'su',
 21: 'al',
 22: 'tío',
 23: 'más',
 24: 'pero',
 25: 'lo',
 26: 'nos',
 27: 'es',
 28: 'sin',
 29: 'para',
 30: 'sus',
 31: 'como',
 32: 'este',
 33: 'era',
 34: 'había',
 35: 'sobre',
 36: 'esta',
 37: 'yo',
 38: 'si',
 39: 'profesor',
 40: 'mis',
 41: 'hans',
 42: 'le',
 43: 'todo',
 44: 'hasta',
 45: 'después',
 46: 'ni',
 47: 'vez',
 48: 'pues',
 49: 'aquel',
 50: 'cuando',
 51: 'dos',
 52: 'bien',
 53: 'ya',
 54: 'tan',
 55: 'mar',
 56: 'o',
 57: 'ha',
 58: 'agua',
 59: 'entonces',
 60: 'cual',
 61: 'qué',
 62: 'tierra',
 63: 'usted',
 64: 'pies',
 65: 'sí',
 66: '¿qué',
 67: 'entre',
 68: 'poco',
 69: 'nuestra',
 70: 'dijo',
 71: 'tiempo',
 72: 'muy',
 73: 'él',
 74: 'ser',
 75: 'hacia',
 76: 'menos',
 77: 'porque',
 78: 'gran',
 79: 'estas',
 80: 'fin',
 81: 'preciso'

In [78]:
# Cantidad de palabras en el vocabulario
vocab_size = len(tok.word_counts)
vocab_size

11372

In [79]:
# El índice para cada palabra
# El sistema las ordena de las más populares a las menos populares
print(tok.word_index)

{'de': 1, 'la': 2, 'que': 3, 'el': 4, 'y': 5, 'a': 6, 'en': 7, '—': 8, 'no': 9, 'un': 10, 'las': 11, 'se': 12, 'los': 13, 'del': 14, 'por': 15, 'mi': 16, 'una': 17, 'con': 18, 'me': 19, 'su': 20, 'al': 21, 'tío': 22, 'más': 23, 'pero': 24, 'lo': 25, 'nos': 26, 'es': 27, 'sin': 28, 'para': 29, 'sus': 30, 'como': 31, 'este': 32, 'era': 33, 'había': 34, 'sobre': 35, 'esta': 36, 'yo': 37, 'si': 38, 'profesor': 39, 'mis': 40, 'hans': 41, 'le': 42, 'todo': 43, 'hasta': 44, 'después': 45, 'ni': 46, 'vez': 47, 'pues': 48, 'aquel': 49, 'cuando': 50, 'dos': 51, 'bien': 52, 'ya': 53, 'tan': 54, 'mar': 55, 'o': 56, 'ha': 57, 'agua': 58, 'entonces': 59, 'cual': 60, 'qué': 61, 'tierra': 62, 'usted': 63, 'pies': 64, 'sí': 65, '¿qué': 66, 'entre': 67, 'poco': 68, 'nuestra': 69, 'dijo': 70, 'tiempo': 71, 'muy': 72, 'él': 73, 'ser': 74, 'hacia': 75, 'menos': 76, 'porque': 77, 'gran': 78, 'estas': 79, 'fin': 80, 'preciso': 81, 'nada': 82, 'superficie': 83, 'axel': 84, 'mismo': 85, 'duda': 86, 'momento': 

In [80]:
# Cantidad de veces quea aparece cada palabra en cada "documento"
# (1 documento = 1 caso de entrada)
print(tok.word_docs)

defaultdict(<class 'int'>, {'centro': 40, 'viaje': 55, 'tierra': 94, 'de': 2191, 'la': 1596, 'al': 402, 'verne': 1, 'julio': 13, '1': 15, 'capítulo': 41, '1863': 2, 'antiguo': 10, 'el': 1250, 'tío': 366, 'en': 1207, 'domingo': 6, 'könig‑strasse': 1, 'profesor': 158, 'hamburgo': 21, 'tradicionales': 1, 'calles': 6, 'hogar': 3, '19': 3, 'entró': 2, 'las': 679, 'a': 1236, 'más': 336, 'barrio': 3, 'su': 413, 'mi': 620, 'situado': 6, '24': 4, 'mayo': 2, 'lidenbrock': 64, 'número': 19, 'del': 641, 'una': 600, 'rápidamente': 17, 'criada': 4, 'comida': 20, 'había': 184, 'apenas': 24, 'empezaba': 4, 'hornillo': 1, 'se': 750, 'sobremanera': 1, 'cocinar': 1, 'excelente': 14, 'preocupó': 1, 'retrasado': 1, 'creyendo': 3, 'que': 1392, 'pues': 119, 'marta': 24, 'va': 13, 'conozco': 3, 'viene': 2, '—': 1065, 'paciencia”': 1, 'con': 547, 'otro': 56, '“bueno”': 1, 'no': 891, 'porque': 77, 'armar': 1, 'hambre': 18, 'hombre': 67, 'mí—': 2, '“si': 2, 'menos': 79, 'para': 253, 'quintín': 1, 'pensé': 14, 's


### Definir el modelo

In [125]:
model = Sequential()

# la primera capa es de embedding entrenable. Recordar que se puede variar el tamaño
# del embedding a entrenar
model.add(Embedding(input_dim=vocab_size+1, output_dim=50, input_shape=(None,)))

model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(100, return_sequences=True))
# model.add(Dense(32, activation='relu'))

# Predicción de clasificación con softmax
# La salida es del tamaño del vocabulario
model.add(Dense(vocab_size+1, activation='softmax'))


# Clasificación multiple categórica --> loss = categorical_crossentropy
# notar que usamos la versión Sparse para utilizar sólo índices en lugar de OHE
model.compile(loss=SparseCategoricalCrossentropy(), optimizer='rmsprop')

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ (None, None, 50)            │         568,650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ (None, None, 100)           │          60,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, None, 100)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ (None, None, 100)           │          80,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, None, 11373)         │       1,148,673 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,858,123 (7.09 MB)

 Trainable params: 1,858,123 (7.09 MB)

 Non-trainable params: 0 (0.00 B)

Dado que por el momento no hay implementaciones adecuadas de la perplejidad que puedan operar en tiempo de entrenamiento, armaremos un Callback *ad-hoc* que la calcule en cada epoch.

**Nota**: un Callback es una rutina gatillada por algún evento, son muy útiles para relevar datos en diferentes momentos del desarrollo del modelo. En este caso queremos hacer un cálculo cada vez que termina una epoch de entrenamiento.

### Entrenamiento

In [126]:
class PplCallback(keras.callbacks.Callback):

    '''
    Este callback es una solución ad-hoc para calcular al final de cada epoch de
    entrenamiento la métrica de Perplejidad sobre un conjunto de datos de validación.
    La perplejidad es una métrica cuantitativa para evaluar la calidad de la generación de secuencias.
    Además implementa la finalización del entrenamiento (Early Stopping)
    si la perplejidad no mejora después de `patience` epochs.
    '''

    def __init__(self, val_data, history_ppl, patience=5):
      # El callback lo inicializamos con secuencias de validación sobre las cuales
      # mediremos la perplejidad y la paciencia para detener el entrenamiento.
      self.val_data = val_data

      self.target = []
      self.padded = []

      count = 0
      self.info = []
      self.history_ppl = []
      self.min_score = np.inf
      self.patience_counter = 0
      self.patience = patience

      # nos movemos en todas las secuencias de los datos de validación
      for seq in self.val_data:

        len_seq = len(seq)
        # armamos todas las subsecuencias
        subseq = [seq[:i] for i in range(len_seq)]
        self.target.extend([seq[i] for i in range(len_seq)])

        if len(subseq)!=0:

          self.padded.append(pad_sequences(subseq, maxlen=max_context_size, padding='pre'))

          self.info.append((count,count+len_seq))
          count += len_seq

      self.padded = np.vstack(self.padded)


    def on_epoch_end(self, epoch, logs=None):

        # en `scores` iremos guardando la perplejidad de cada secuencia
        scores = []

        # Predict in smaller batches to reduce memory usage
        batch_size = 16  # Experiment with different batch sizes
        for i in range(0, len(self.padded), batch_size):
          batch_padded = self.padded[i:i+batch_size]
          predictions = self.model.predict(batch_padded, verbose=0)

          # para cada secuencia de validación
          for k, (start, end) in enumerate(self.info):
            if start >= i and end <= i + batch_size:  # Only process sequences in the current batch
              # en `probs` iremos guardando las probabilidades de los términos target
              # Calculate the index within the batch. Use max to avoid negative indices
              j = max(0, k - i // batch_size * batch_size)
              probs = [predictions[j,-1,idx_vocab] for idx_seq, idx_vocab in zip(range(start-i, end-i),self.target[start:end])]

              # calculamos la perplejidad por medio de logaritmos
              scores.append(np.exp(-np.sum(np.log(probs))/(end-start)))

        # promediamos todos los scores e imprimimos el valor promedio
        current_score = np.mean(scores)
        history_ppl.append(current_score)
        print(f'\n mean perplexity: {current_score} \n')

        # chequeamos si tenemos que detener el entrenamiento
        if current_score < self.min_score:
          self.min_score = current_score
          self.model.save("my_model.h5")
          print("Saved new model!")
          self.patience_counter = 0
        else:
          self.patience_counter += 1
          if self.patience_counter == self.patience:
            print("Stopping training...")
            self.model.stop_training = True

In [127]:
# fiteamos, nótese el agregado del callback con su inicialización. El batch_size lo podemos seleccionar a mano
# en general, lo mejor es escoger el batch más grande posible que minimice el tiempo de cada época.
# En la variable `history_ppl` se guardarán los valores de perplejidad para cada época.
history_ppl = []
hist = model.fit(X, y, epochs=30, callbacks=[PplCallback(tokenized_sentences_val,history_ppl, patience=10)], batch_size=32)

Epoch 1/30
208/211 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 7.0511


 mean perplexity: 5127.773646405815 

Saved new model!
211/211 ━━━━━━━━━━━━━━━━━━━━ 69s 323ms/step - loss: 7.0346
Epoch 2/30
210/211 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 5.7068
 mean perplexity: 5378.92863346398 

211/211 ━━━━━━━━━━━━━━━━━━━━ 92s 369ms/step - loss: 5.7063
Epoch 3/30
209/211 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 5.6118
 mean perplexity: 5755.707940784024 

211/211 ━━━━━━━━━━━━━━━━━━━━ 73s 329ms/step - loss: 5.6120
Epoch 4/30
210/211 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 5.6446
 mean perplexity: 5858.873071263016 

211/211 ━━━━━━━━━━━━━━━━━━━━ 82s 328ms/step - loss: 5.6444
Epoch 5/30
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 5.6050
 mean perplexity: 5895.429619761869 

211/211 ━━━━━━━━━━━━━━━━━━━━ 86s 346ms/step - loss: 5.6051
Epoch 6/30
209/211 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 5.5734
 mean perplexity: 6216.60143522286 

211/211 ━━━━━━━━━━━━━━━━━━━━ 70s 335ms/step - loss: 5.5740
Epoch 7/30
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 5

In [128]:
import matplotlib.pyplot as plt
import seaborn as sns

# Podemos graficar la evolución de la perplejidad con las épocas.
# Recordar que el valor de perplejidad del modelo trivial es el tamaño del vocabulario.
epoch_count = range(1, len(history_ppl) + 1)
sns.lineplot(x=epoch_count,  y=history_ppl)
plt.show()

In [129]:
# Cargamos el mejor modelo guardado del entrenamiento para hacer inferencia
model = keras.models.load_model('my_model.h5')


### Predicción de próxima palabra

In [130]:
# Se puede usar gradio para probar el modelo
# Gradio es una herramienta muy útil para crear interfaces para ensayar modelos
# https://gradio.app/

!pip install -q gradio

In [131]:
import gradio as gr

def model_response(human_text):

    # Encodeamos
    encoded = tok.texts_to_sequences([human_text])[0]
    # Si tienen distinto largo
    encoded = pad_sequences([encoded], maxlen=max_context_size, padding='pre')

    # Predicción softmax
    y_hat = np.argmax(model.predict(encoded)[0,-1,:])


    # Debemos buscar en el vocabulario la palabra
    # que corresopnde al indice (y_hat) predicho por le modelo
    out_word = ''
    for word, index in tok.word_index.items():
        if index == y_hat:
            out_word = word
            break

    # Agrego la palabra a la frase predicha
    return human_text + ' ' + out_word

iface = gr.Interface(
    fn=model_response,
    inputs=["textbox"],
    outputs="text")

iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://6c12c173b8294e400d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6c12c173b8294e400d.gradio.live


### Generación de secuencias

In [132]:
def generate_seq(model, tokenizer, seed_text, max_length, n_words):
    """
        Exec model sequence prediction

        Args:
            model (keras): modelo entrenado
            tokenizer (keras tokenizer): tonenizer utilizado en el preprocesamiento
            seed_text (string): texto de entrada (input_seq)
            max_length (int): máxima longitud de la sequencia de entrada
            n_words (int): números de palabras a agregar a la sequencia de entrada
        returns:
            output_text (string): sentencia con las "n_words" agregadas
    """
    output_text = seed_text
	  # generate a fixed number of words
    for _ in range(n_words):
		  # Tokenizamos
      encoded = tokenizer.texts_to_sequences([output_text])[0]

		  # Si tienen distinto largo
      encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')

		  # Predicción softmax
      y_hat = np.argmax(model.predict(encoded)[0,-1,:])

      # Vamos concatenando las predicciones
      out_word = ''

      # Debemos buscar en el vocabulario la palabra
      # que corresopnde al indice (y_hat) predicho por le modelo
      for word, index in tokenizer.word_index.items():
        if index == y_hat:
          out_word = word
          break

		  # Agrego las palabras a la frase predicha
      output_text += ' ' + out_word

    return output_text

In [133]:
input_text='La cabeza me daba vueltas'

generate_seq(model, tok, input_text, max_length=max_context_size, n_words=10)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


'La cabeza me daba vueltas          '

###  Beam search y muestreo aleatorio

In [134]:
# funcionalidades para hacer encoding y decoding

def encode(text,max_length=max_context_size):

    encoded = tok.texts_to_sequences([text])[0]
    encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')

    return encoded

def decode(seq):
    return tok.sequences_to_texts([seq])

In [135]:
from scipy.special import softmax

# función que selecciona candidatos para el beam search
def select_candidates(pred,num_beams,vocab_size,history_probs,history_tokens,temp,mode):

  # colectar todas las probabilidades para la siguiente búsqueda
  pred_large = []

  for idx,pp in enumerate(pred):
    pred_large.extend(np.log(pp+1E-10)+history_probs[idx])

  pred_large = np.array(pred_large)

  # criterio de selección
  if mode == 'det':
    idx_select = np.argsort(pred_large)[::-1][:num_beams] # beam search determinista
  elif mode == 'sto':
    idx_select = np.random.choice(np.arange(pred_large.shape[0]), num_beams, p=softmax(pred_large/temp)) # beam search con muestreo aleatorio
  else:
    raise ValueError(f'Wrong selection mode. {mode} was given. det and sto are supported.')

  # traducir a índices de token en el vocabulario
  new_history_tokens = np.concatenate((np.array(history_tokens)[idx_select//vocab_size],
                        np.array([idx_select%vocab_size]).T),
                      axis=1)

  # devolver el producto de las probabilidades (log) y la secuencia de tokens seleccionados
  return pred_large[idx_select.astype(int)], new_history_tokens.astype(int)


def beam_search(model,num_beams,num_words,input,temp=1,mode='det'):

    # first iteration

    # encode
    encoded = encode(input)

    # first prediction
    y_hat = model.predict(encoded,verbose=0)[0,-1,:]

    # get vocabulary size
    vocab_size = y_hat.shape[0]

    # initialize history
    history_probs = [0]*num_beams
    history_tokens = [encoded[0]]*num_beams

    # select num_beams candidates
    history_probs, history_tokens = select_candidates([y_hat],
                                        num_beams,
                                        vocab_size,
                                        history_probs,
                                        history_tokens,
                                        temp,
                                        mode)

    # beam search loop
    for i in range(num_words-1):

      preds = []

      for hist in history_tokens:

        # actualizar secuencia de tokens
        input_update = np.array([hist[i+1:]]).copy()

        # predicción
        y_hat = model.predict(input_update,verbose=0)[0,-1,:]

        preds.append(y_hat)

      history_probs, history_tokens = select_candidates(preds,
                                                        num_beams,
                                                        vocab_size,
                                                        history_probs,
                                                        history_tokens,
                                                        temp,
                                                        mode)

    return history_tokens

In [136]:
# predicción con beam search.
# Se pueden camiar los modos entre 'det' (determinista) y
# 'sto' (estocástico)
# para el caso estocástico también se puede variar la temperatura
salidas = beam_search(model,num_beams=10,num_words=6,input="La cabeza me daba vueltas",temp=1,mode='sto')

In [137]:
# tenemos `num_beams` salidas ordenadas de mayor a menor likelihood
salidas.shape

(10, 40)

In [138]:
# veamos las salidas
decode(salidas[0])

['la cabeza me daba vueltas si']

### Conclusiones:

Debido a la cantidad de oraciones, no se podía utilizar el colab correctamente por falta de memoria al momento de realizar el entrenamieto. Por eso no se aumentó el volumen de oraciones lo que causó que el entrenamiento no haya salido tan bien como en el caso de los caracteres.